In [2]:
from CoFiPruning.models.l0_module import L0Module
import torch
from transformers import AutoModelForCausalLM

# a = torch.load("/scratch/gpfs/mengzhou/space2/out/test/CoFi_opt-125m_Books3/checkpoint-500/l0_module.pt", map_location=torch.device('cpu'))
b = torch.load("/scratch/gpfs/mengzhou/space2/out/test/CoFi_opt-125m_Books3/checkpoint-500/pytorch_model.bin", map_location=torch.device('cpu'))
c = AutoModelForCausalLM.from_pretrained("facebook/opt-125m") 

In [3]:
b["model.decoder.layers.0.self_attn.out_proj.weight"] == c.model.decoder.layers[0].self_attn.out_proj.weight


tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]])

In [15]:
b["model.decoder.embed_tokens.weight"]

tensor([[ 0.1148, -0.1440,  0.0555,  ...,  0.2147,  0.0835,  0.0668],
        [ 0.1148, -0.1440,  0.0547,  ...,  0.2146,  0.0835,  0.0668],
        [-0.0022, -0.0893,  0.0993,  ..., -0.0376,  0.0096, -0.1073],
        ...,
        [ 0.1150, -0.1439,  0.0548,  ...,  0.2151,  0.0834,  0.0670],
        [ 0.1149, -0.1446,  0.0559,  ...,  0.2146,  0.0835,  0.0669],
        [ 0.1146, -0.1431,  0.0553,  ...,  0.2132,  0.0836,  0.0671]])

In [16]:
c.model.decoder.embed_tokens.weight

Parameter containing:
tensor([[ 0.1150, -0.1438,  0.0555,  ...,  0.2146,  0.0833,  0.0669],
        [ 0.1149, -0.1438,  0.0547,  ...,  0.2145,  0.0833,  0.0669],
        [ 0.0010, -0.0922,  0.1025,  ..., -0.0402,  0.0060, -0.1078],
        ...,
        [ 0.1152, -0.1437,  0.0547,  ...,  0.2145,  0.0833,  0.0671],
        [ 0.1151, -0.1455,  0.0546,  ...,  0.2156,  0.0837,  0.0673],
        [ 0.1156, -0.1437,  0.0577,  ...,  0.2139,  0.0833,  0.0650]],
       requires_grad=True)

In [6]:
from CoFiPruning.models.modeling_opt import CoFiOPTForCausalLM

In [7]:
m = CoFiOPTForCausalLM.from_pretrained("facebook/opt-125m")

In [9]:
m.state_dict().keys()

odict_keys(['model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'model.decoder.final_layer_norm.weight', 'model.decoder.final_layer_norm.bias', 'model.decoder.layers.0.self_attn.k_proj.weight', 'model.decoder.layers.0.self_attn.k_proj.bias', 'model.decoder.layers.0.self_attn.v_proj.weight', 'model.decoder.layers.0.self_attn.v_proj.bias', 'model.decoder.layers.0.self_attn.q_proj.weight', 'model.decoder.layers.0.self_attn.q_proj.bias', 'model.decoder.layers.0.self_attn.out_proj.weight', 'model.decoder.layers.0.self_attn.out_proj.bias', 'model.decoder.layers.0.self_attn_layer_norm.weight', 'model.decoder.layers.0.self_attn_layer_norm.bias', 'model.decoder.layers.0.fc1.weight', 'model.decoder.layers.0.fc1.bias', 'model.decoder.layers.0.fc2.weight', 'model.decoder.layers.0.fc2.bias', 'model.decoder.layers.0.final_layer_norm.weight', 'model.decoder.layers.0.final_layer_norm.bias', 'model.decoder.layers.1.self_attn.k_proj.weight', 'model.decoder.layers.1.self_attn.k_p

In [ ]:
# test how to can fit a 13b model to a single GPU
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("facebook/opt-13b")
model.cuda().half()

In [ ]:
import nvidia_smi
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
# card id 0 hardcoded here, there is also a call to get all available card ids, so we could iterate
info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)
nvidia_smi.nvmlShutdown()

# an alternative 
print(torch.cuda.memory_summary())

# about the difference between torch.cuda.memory_summary() and nvidia-smi
"""Yes, calling .half() on the model as well as the inputs would give you an estimate of how much memory would be saved in the extreme case of using float16 for all operations. Make sure to check the memory usage via torch.cuda.memory_summary(), since nvidia-smi will also show the cache not only the allocated memory."""
# https://discuss.pytorch.org/t/using-half-precision/94995/4




In [5]:
import os
for ts in [0.3, 0.6, 0.9]:
    for n in [1000, 5000, 10000]:
        for lr in [0.01, 0.1, 1.0]:
        
            f = f"/scratch/gpfs/mengzhou/space2/out/test_round1_datanum_reglr_sparsity/CoFi_{n}_{lr}_{ts}_opt-1.3b/log.txt"
            if not os.path.exists(f): # or not os.path.exists(os.path.dirname(f) + "/pytorch_model.bin"):
                continue
            
            lines = open(f, "r").readlines()
            num = []
            for i, line in enumerate(lines):
                if "Eval @ " in line:
                    num.append(i) 

            num = num[-4]

            count = 0
            sparsity = 0
            ppls = []
            for line in lines[num:]:
                # if "sparsity:" in line and "target" not in line:
                if "ppl" in line:
                    ppls.append(line.strip().split(" ")[-1])
                    count += 1
                if "sparsity" in line and sparsity == 0:
                    sparsity = line.strip().split(" ")[-1]
                if count == 4:
                    break
            print(f"{n} {lr} {ts} {sparsity} {ppls[0]} {ppls[1]} {ppls[2]} {ppls[3]}")

1000 0.01 0.3 0.0053 20.1754 4.748 8.5589 10.7826
1000 0.1 0.3 0.4979 100.9221 39.0003 56.7808 66.9948
1000 1.0 0.3 0.0609 23.8185 5.9713 10.4617 14.0719
5000 0.01 0.3 0.3493 32.4293 11.5003 20.6394 25.4921
5000 0.1 0.3 0.0735 22.7757 5.834 10.3339 12.9699
5000 1.0 0.3 0.1798 26.5311 7.1342 12.2748 16.6136
10000 0.01 0.3 0.4935 38.7686 1.0} 12.6062 1.0}
10000 0.1 0.3 0.4914 62.5191 20.2766 33.0188 50.7191
10000 1.0 0.3 0.2909 32.527 8.619 15.6006 21.3937
1000 0.01 0.6 0.0053 20.1622 4.7454 8.5522 10.7975
1000 0.1 0.6 0.5772 138.3106 62.3504 83.3525 114.8327
1000 1.0 0.6 0.2641 43.4385 12.6153 20.7291 27.8756
5000 0.01 0.6 0.0129 10.873 0.5} 20.645 0.5}
5000 0.1 0.6 0.7593 146.1183 69.7352 95.0185 165.1519
5000 1.0 0.6 0.6864 208.9983 99.2549 118.4172 237.3473
10000 0.01 0.6 0.0 20.0163 4.6954 8.5148 10.6908
10000 0.1 0.6 0.7847 161.7515 60.8145 88.2342 166.8606
10000 1.0 0.6 0.6866 93.6509 34.2631 52.9171 91.4273
1000 0.01 0.9 0.0053 20.1545 4.7376 8.5548 10.7979
1000 0.1 0.9 0.6354 24

In [34]:
n = 10000
lr = 1.0
ts = 0.6

f = f"/scratch/gpfs/mengzhou/space2/out/test_round1_datanum_reglr_sparsity/CoFi_{n}_{lr}_{ts}_opt-1.3b/trainer_state.json"

    
import json
state = json.load(open(f, "r"))
sparsity = []
ppls = []
domain = "Books3"
key = "eval_Books3-valid-1.536M_ppl"
for log in state["log_history"][::4]:
    if key in log:
        sparsity.append(log["sparsity"])
        ppls.append(log[key])


In [35]:
print("sparsity every 50 steps:", " ".join(str(round(s, 2)) for s in sparsity))
print("ppl every 50 steps:", " ".join(str(round(s, 2)) for s in ppls))

sparsity every 50 steps: 0.05 0.07 0.67 0.77 0.8 0.79 0.61 0.56 0.53
ppl every 50 steps: 22.35 22.44 294.89 302.55 309.64 297.63 119.76 85.3 68.13


1000 0.1 0.3 0.3 100.9221 39.0003 56.7808 66.9948
1000 0.1 0.6 0.6 138.3106 62.3504 83.3525 114.8327
1000 1.0 0.3 0.3 23.8185 5.9713 10.4617 14.0719
1000 1.0 0.6 0.6 43.4385 12.6153 20.7291 27.8756
1000 1.0 0.9 0.9 1295.5627 1160.6124 869.1015 1521.6621